In [1]:
import pandas as pd
import numpy as np
import model_On_Array2
import os
from datetime import datetime, timedelta
import process_utils
import itertools
import dataprocessing2
from datetime import datetime, timedelta
import itertools
from sklearn.model_selection import train_test_split
import scipy
from scipy.integrate import odeint

In [2]:
model=model_On_Array2.mainmodel(maison='multiroom1')

In [3]:
model.update_theta()

In [4]:
model.set_theta(model.theta)

In [5]:
model.setupdata()

In [6]:
test=model.raw.to_numpy()

In [7]:
df=model.raw

In [10]:
df

data                            T                                           \
zone                     bathroom bedroom_1 bedroom_2 bedroom_3 diningroom   
uniqueday_Id intraday_Id                                                     
2020-05-24   19:40:03        21.8      22.5      22.4      21.7       22.0   
             19:45:03        21.8      22.5      22.4      21.7       22.0   
             19:50:03        21.8      22.4      22.4      21.7       21.7   
             19:55:03        21.8      22.4      22.4      21.7       21.7   
             20:00:04        21.8      22.4      22.4      21.7       21.7   
...                           ...       ...       ...       ...        ...   
2021-05-24   16:45:04        19.5      19.0      19.3      20.3       20.1   
             16:50:03        19.5      19.0      19.3      20.3       20.1   
             16:55:03        19.4      19.0      19.3      20.3       20.1   
             17:00:04        19.4      19.0      19.3      20.3       20.1   
             17:05:03        19.4      19.0      19.3      20.3       20.3   

data                                            Tset                      ...  \
zone                     kitchen livingroom bathroom bedroom_1 bedroom_2  ...   
uniqueday_Id intraday_Id                                                  ...   
2020-05-24   19:40:03       22.0       22.9     16.0      16.0      16.0  ...   
             19:45:03       22.0       22.9     16.0      16.0      16.0  ...   
             19:50:03       22.0       22.9     16.0      16.0      16.0  ...   
             19:55:03       22.0       22.9     16.0      16.0      16.0  ...   
             20:00:04       22.0       22.9     16.0      16.0      16.0  ...   
...                          ...        ...      ...       ...       ...  ...   
2021-05-24   16:45:04       20.5       18.7     16.0      15.0      15.0  ...   
             16:50:03       20.5       18.7     16.0      15.0      15.0  ...   
             16:55:03       20.5       18.7     16.0      15.0      15.0  ...   
             17:00:04       20.8       18.7     16.0      15.0      15.0  ...   
             17:05:03       20.8       18.7     16.0      15.0      15.0  ...   

data                          Tnext       Tset sequence_Id      Rad            \
zone                     singleroom singleroom             bathroom bedroom_1   
uniqueday_Id intraday_Id                                                        
2020-05-24   19:40:03       21.1625  15.857143           0    False     False   
             19:45:03       21.1125  15.857143           0    False     False   
             19:50:03       21.1125  15.857143           0    False     False   
             19:55:03       21.1125  15.857143           0    False     False   
             20:00:04       21.1125  15.857143           0    False     False   
...                             ...        ...         ...      ...       ...   
2021-05-24   16:45:04       18.8000  16.714286           0    False     False   
             16:50:03       18.7875  16.714286           0    False     False   
             16:55:03       18.8250  16.714286           0    False     False   
             17:00:04       18.8500  16.714286           0    False     False   
             17:05:03       18.8500  16.714286           0    False     False   

data                                                                        
zone                     bedroom_2 bedroom_3 diningroom kitchen livingroom  
uniqueday_Id intraday_Id                                                    
2020-05-24   19:40:03        False     False      False   False      False  
             19:45:03        False     False      False   False      False  
             19:50:03        False     False      False   False      False  
             19:55:03        False     False      False   False      False  
             20:00:04        False     False      False   False      False  
...                  

In [295]:
mydata={'state':np.empty((10000,288,7)),'Text':np.ones((10000,288,1)),'timestamp':0.05*np.ones((100,288))}

In [313]:
mydata['state'][0][0]=np.ones(7)/7

T=np.random.rand(8,8)@np.concatenate((mydata['state'][0][0],mydata['Text'][0][0]))

In [329]:
def thermalT(state,Text):
    T=np.random.rand(8,8)@np.concatenate((state,Text))
    return T[:-1]

In [349]:
thermalT([i for i in range(7)],[1])

array([ 9.29721913,  8.83415811, 13.07461107, 11.63635476, 11.86720758,
       12.71355562, 12.80349462])

In [333]:
thermalT(mydata['state'][0][0],mydata['Text'][0][0])

array([1.22970435, 0.75602013, 1.50883808, 0.59407142, 1.15213874,
       1.27019551, 0.69105396])

In [406]:
def simulate(state,Text):
    Th = np.empty_like(state)
    Th=np.append((Text[0]+5)*[np.ones(7)],Th,axis=0)
    for i,out in enumerate(Text):
        Th[i+1]=thermalT(state[i],out)
    return Th[1:]
    

In [1]:
mydata['state'][0]

NameError: name 'mydata' is not defined

In [408]:
test=simulate(mydata['state'][0],mydata['Text'][0])

In [415]:
len(mydata['state'])

10000

In [420]:
test=[simulate(mydata['state'][i],mydata['Text'][i]) for i in range(200)]

In [418]:
test[0]

array([[1.33316925, 0.91880576, 1.33122348, ..., 1.18195188, 0.62184465,
        1.61003864],
       [0.63781354, 0.03950834, 0.82094348, ..., 0.39925883, 0.53857943,
        0.79132249],
       [0.6930081 , 0.0577304 , 0.22652246, ..., 0.99525855, 0.21640336,
        0.31451342],
       ...,
       [0.78685552, 0.1382189 , 0.57728547, ..., 0.02910281, 0.1613586 ,
        0.53728538],
       [0.11007829, 0.33529494, 0.89013129, ..., 0.23984261, 0.0883408 ,
        0.77436914],
       [0.96767066, 0.67185636, 0.50775077, ..., 0.13619449, 0.89385052,
        0.13008202]])

In [323]:
thermalT(mydata['state'][0][0],mydata['Text'][0][0])

array([1.14285714, 1.14285714, 1.14285714, 1.14285714, 1.14285714,
       1.14285714, 1.14285714])

partie ODE (transformer timestamp en fonction)

In [306]:
def func(state,t,mycoeff):
    T=state+
    return state+T[:-1]

In [312]:
transition(mydata['state'][0][0],mydata['Text'][0][0])

array([0.96571257, 1.70055798, 1.02363828, 1.66491054, 1.74604356,
       0.92551689, 1.74049707])

In [278]:
def deriv(y,t):
    #dsdt = np.random.rand(7,7)@state
    dydt=
    return dydt
y0=[0,0,0,0,0,0,0]
t = np.linspace(0, 10, 101)
odeint(deriv,y0,mydata['timestamp'][0],args=())

SyntaxError: invalid syntax (40535140.py, line 3)